In [98]:
# mlp for combined regression and classification predictions on the abalone dataset
from numpy import unique
from numpy import argmax
from pandas import read_csv
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import plot_model
import tensorflow as tf

In [1]:
import pandas as pd

df = pd.read_csv('dataset/TrainingAnnotation.csv')
df = df.dropna()

# Remove unused data
df = df.drop(columns=['PED'], axis=1)
df.head(5)

,patient ID,gender,age,diagnosis,preVA,anti-VEGF,preCST,preIRF,preSRF,prePED,preHRF,VA,continue injection,CST,IRF,SRF,HRF
2,0000-0673L,2,67,2,2.00,1,373.0,0.0,1.0,1.0,0.0,2.0,1.0,381.0,0.0,1.0,1.0
3,0000-1315L,2,76,6,0.30,0,227.0,0.0,0.0,0.0,0.0,0.4,0.0,237.0,0.0,0.0,0.0
4,0000-0656L,1,64,2,0.32,1,201.0,0.0,1.0,1.0,0.0,0.4,1.0,191.0,0.0,0.0,0.0
5,0000-0656R,1,64,2,0.30,0,282.0,0.0,0.0,0.0,0.0,0.1,0.0,202.0,0.0,0.0,0.0
6,0000-1792L,2,52,6,0.12,0,198.0,0.0,1.0,0.0,0.0,0.5,0.0,230.0,0.0,0.0,0.0


gonna try multilable multioutput approach, first choose desired label and target
honestly VA range are not even close to CST value, but let's try group them first as regression label for now
then continue injection, IRF, SRF, and HRF are the class label

and feature, not quite sure, because there are varying type, age have integer value, gender diagnosis anti-VGEF are classes, and preVA are regression
but since model won't have problem with that, let's just keep it that way and make their data type all float

In [28]:
label_binary = ['continue injection', 'IRF', 'SRF', 'HRF']
label_regression = ['preCST','VA','CST']
feature = ['gender', 'age', 'diagnosis', 'preVA', 'anti-VEGF']

In [115]:
X, y_binary, y_reg = df[feature].astype('float'), df[label_binary], df[label_regression]
X_train, X_test, y_train_bin, y_test_bin, y_train_reg, y_test_reg = train_test_split(X,y_binary,y_reg, test_size=0.33, shuffle=False)
n_features = X.shape[1]
n_binary_label = y_train_bin.shape[1]
n_regression_label = y_train_reg.shape[1]

model are pretty simple, it's like two-headed snake

In [116]:
visible = Input(shape=(n_features,))
hidden1 = Dense(20, activation='relu', kernel_initializer='he_normal')(visible)
hidden2 = Dense(10, activation='relu', kernel_initializer='he_normal')(hidden1)
# binary output
out_binary = Dense(n_binary_label, activation='sigmoid', name='binary')(hidden2)
# regression output
out_reg = Dense(n_regression_label, activation='linear', name='regression')(hidden2)

# define model
model = Model(inputs=visible, outputs=[out_binary,out_reg])
model.compile(loss=['binary_crossentropy','mse'], optimizer='adam')
model.fit(X_train, [y_train_bin,y_train_reg], epochs=300, batch_size=32, verbose=2)

Epoch 1/300
46/46 - 1s - loss: 124794.4922 - binary_loss: 16.0970 - regression_loss: 124778.3750 - 505ms/epoch - 11ms/step
Epoch 2/300
46/46 - 0s - loss: 111528.4141 - binary_loss: 6.6059 - regression_loss: 111521.8047 - 38ms/epoch - 826us/step
Epoch 3/300
46/46 - 0s - loss: 97857.1328 - binary_loss: 7.3342 - regression_loss: 97849.8203 - 42ms/epoch - 913us/step
Epoch 4/300
46/46 - 0s - loss: 82385.1406 - binary_loss: 10.3926 - regression_loss: 82374.7500 - 41ms/epoch - 891us/step
Epoch 5/300
46/46 - 0s - loss: 65870.7891 - binary_loss: 9.7929 - regression_loss: 65860.9922 - 40ms/epoch - 870us/step
Epoch 6/300
46/46 - 0s - loss: 50465.7031 - binary_loss: 4.6642 - regression_loss: 50461.0391 - 45ms/epoch - 978us/step
Epoch 7/300
46/46 - 0s - loss: 37140.7773 - binary_loss: 1.2110 - regression_loss: 37139.5703 - 43ms/epoch - 935us/step
Epoch 8/300
46/46 - 0s - loss: 28647.3633 - binary_loss: 0.6533 - regression_loss: 28646.7031 - 48ms/epoch - 1ms/step
Epoch 9/300
46/46 - 0s - loss: 25215

In [148]:
# plot_model(model, to_file='model.png', show_shapes=True)
# not working, the error said must instasll pydot and graphviz, even if installed already it wont working
model.summary()

Model: "model_11"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_13 (InputLayer)          [(None, 5)]          0           []                               
                                                                                                  
 dense_30 (Dense)               (None, 20)           120         ['input_13[0][0]']               
                                                                                                  
 dense_31 (Dense)               (None, 10)           210         ['dense_30[0][0]']               
                                                                                                  
 binary (Dense)                 (None, 4)            44          ['dense_31[0][0]']               
                                                                                           

sadly regression loss was big, as expected, let's see and evaluate using test data

In [117]:
yhat_bin, yhat_reg = model.predict(X_test)

0.70288324

take a peek

In [118]:
yhat_bin_df = pd.DataFrame(yhat_bin)
print(yhat_bin_df)
print(y_test_bin)

            0         1         2         3
0    0.786965  0.528570  0.354183  0.520885
1    0.473601  0.438832  0.261749  0.317501
2    0.163457  0.071898  0.068966  0.080764
3    0.319097  0.423497  0.231856  0.278576
4    0.190542  0.386887  0.197827  0.209070
..        ...       ...       ...       ...
706  0.701509  0.997954  0.941984  0.899048
707  0.120242  0.065954  0.061850  0.069104
708  0.332899  0.413879  0.232618  0.253212
709  0.285083  0.069067  0.072982  0.140120
710  0.488078  0.407779  0.242560  0.391434

[711 rows x 4 columns]
      continue injection  IRF  SRF  HRF
1576                 1.0  1.0  1.0  1.0
1577                 0.0  0.0  1.0  1.0
1578                 0.0  0.0  0.0  0.0
1579                 1.0  1.0  0.0  0.0
1580                 0.0  0.0  0.0  1.0
...                  ...  ...  ...  ...
2360                 0.0  0.0  0.0  0.0
2361                 0.0  0.0  0.0  0.0
2363                 1.0  1.0  0.0  0.0
2364                 0.0  0.0  0.0  0.0
2365    

let's see each label confusion matrixs

In [139]:
import numpy as np
from sklearn.metrics import multilabel_confusion_matrix

multilabel_confusion_matrix(y_test_bin, yhat_bin>0.5)

array([[[345, 112],
        [121, 133]],

       [[430,  65],
        [171,  45]],

       [[541,  30],
        [137,   3]],

       [[460,  67],
        [142,  42]]], dtype=int64)

now let's measure each label F1 score, precision, and recall

In [143]:
from sklearn.metrics import classification_report
print(classification_report(
    y_test_bin,
    yhat_bin>0.5,
    target_names=['continue injection', 'IRF','SRF','HRF']
))

                    precision    recall  f1-score   support

continue injection       0.54      0.52      0.53       254
               IRF       0.41      0.21      0.28       216
               SRF       0.09      0.02      0.03       140
               HRF       0.39      0.23      0.29       184

         micro avg       0.45      0.28      0.35       794
         macro avg       0.36      0.25      0.28       794
      weighted avg       0.39      0.28      0.32       794
       samples avg       0.16      0.16      0.14       794

